In [103]:
!pip install tweepy

In [104]:
import sys
sys.executable

'c:\\Users\\14708\\AppData\\Local\\Programs\\Python\\Python311\\python.exe'

In [105]:
import numpy as np 
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px 
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

pd.set_option('display.max_columns', 25)

import tweepy

ModuleNotFoundError: No module named 'tweepy'

In [ ]:
main_df=pd.read_csv(r"C:\Users\14708\Downloads\Tata and Hyundai cars - data.xlsx - Data.csv")
df = main_df.copy()

In [ ]:
df.head()

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,Reach,Desktop Reach,Mobile Reach,Twitter Social Echo,Facebook Social Echo,Reddit Social Echo,National Viewership,Engagement,Sentiment,Keywords,State
0,28-Sep-2023 02:08PM,Kia और Hyundai ने वापस बुलाईं अपनी 35 लाख कारे...,https://hindi.pardaphash.com/kia-and-hyundai-r...,Kia and Hyundai Recalled Cars: वाहन निर्माता क...,Kia and Hyundai Recalled Cars: वाहन निर्माता क...,Pardaphash,NaN,India,Delhi,Hindi,31307.0,1279.0,30028.0,0.0,0.0,0.0,0,NaN,Neutral,"Hyundai,Cars",Delhi
1,28-Sep-2023 01:46PM,"Tata Punch EV spied, may compete with MG Comen...",https://economictimes.indiatimes.com/industry/...,Tata Motors is reportedly developing an electr...,... Punch EV is expected to compete with the M...,The Economic Times,NaN,India,National,English,15403059.0,6171732.0,9231327.0,0.0,0.0,0.0,0,NaN,Neutral,"Tata,cars",Maharashtra
2,28-Sep-2023 01:30PM,Tata Avinya Reviews On Road Price 2023 Mileage...,https://badisoch.in/automobile-news/tata-aviny...,... (adsbygoogle = window.adsbygoogle || []).p...,"... , despite most carmakers focusing on large...",Badi Soch,NaN,India,NaN,English,1128280.0,171549.0,956731.0,0.0,0.0,0.0,0,NaN,Neutral,"cars,Tata",NaN
3,28-Sep-2023 01:24PM,NaN,http://twitter.com/automobilindia8/statuses/17...,NaN,Engine Fire-Related Risks Force Kia & Hyundai ...,Twitter,@automobilindia8,India,NaN,English,2.0,NaN,NaN,NaN,NaN,NaN,0,0.0,Neutral,"Hyundai,Cars,hyundai",Karnataka
4,28-Sep-2023 01:22PM,NaN,http://twitter.com/MalakpetD/statuses/17073021...,NaN,"@srinualavilli @BMTC_BENGALURU Mahindra, Tata,...",Twitter,@MalakpetD,India,NaN,English,506.0,NaN,NaN,NaN,NaN,NaN,0,0.0,Neutral,"Tata,cars",Telangana


In [ ]:
df.tail()

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,Reach,Desktop Reach,Mobile Reach,Twitter Social Echo,Facebook Social Echo,Reddit Social Echo,National Viewership,Engagement,Sentiment,Keywords,State
1523,12-Sep-2023 06:20AM,MG3 hatchback takes out ‘cheapest car to run’ ...,https://bulletinreporter.com/mg3-hatchback-tak...,A small car has claimed the title of the cheap...,"... Costs Survey, which lists the cost of runn...",Bulletin Reporter,Kemal Pasha,India,NaN,English,2074.0,2074.0,0.0,0.0,0.0,0.0,0,NaN,Positive,"Hyundai,cars",NaN
1524,12-Sep-2023 05:24AM,MG3 hatchback named by RACV as the most cost-e...,https://vervetimes.com/mg3-hatchback-named-by-...,A small car has claimed the title of the cheap...,"... Costs Survey, which lists the cost of runn...",Verve times - Latest News around the World,NaN,India,NaN,English,26425.0,7491.0,18934.0,0.0,0.0,0.0,0,NaN,Positive,"Hyundai,cars",NaN
1525,12-Sep-2023 04:37AM,RACV names MG3 hatchback as the cheapest vehic...,https://newsconcerns.com/racv-names-mg3-hatchb...,A small car has claimed the title of the cheap...,"... Costs Survey, which lists the cost of runn...",News Concerns - Latest News Update,Charlie Brown,India,Madhya Pradesh,English,9323.0,2527.0,6796.0,0.0,0.0,0.0,0,NaN,Positive,"Hyundai,cars",Madhya Pradesh
1526,12-Sep-2023 04:04AM,Tata Power EZ App Electrifies Green Revolution...,https://thenfapost.com/tata-power-ez-app-elect...,Tata Power EZ App Electrifies Green Revolution...,Charging hubs for leading cab fleet operators....,NFAPost,N V Vijayakumar,India,Bangalore,English,851.0,851.0,0.0,0.0,0.0,0.0,0,NaN,Neutral,"Tata,cars",Karnataka
1527,12-Sep-2023 02:44AM,NaN,http://twitter.com/poojashah0998/statuses/1701...,NaN,RT @ShamsNaqvi: One of the most awaited cars t...,Twitter,@poojashah0998,India,NaN,English,45.0,NaN,NaN,NaN,NaN,NaN,0,NaN,Positive,"cars,Tata",Maharashtra


In [ ]:
df.sample(5)

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,Reach,Desktop Reach,Mobile Reach,Twitter Social Echo,Facebook Social Echo,Reddit Social Echo,National Viewership,Engagement,Sentiment,Keywords,State
219,25-Sep-2023 08:21AM,NaN,http://twitter.com/Ni3iN_D28/statuses/17061392...,NaN,RT @gaadiwaadi: 10+ Highly Anticipated Cars Co...,Twitter,@Ni3iN_D28,India,NaN,English,276.0,NaN,NaN,NaN,NaN,NaN,0,NaN,Positive,"Cars,Tata",Maharashtra
1188,13-Sep-2023 11:20AM,NaN,https://twitter.com/Gokul_yt_/statuses/1701835...,NaN,RT @AVinthehousee: #ThalapathyVijay is a Car E...,Twitter,@gokul_yt_,India,NaN,English,141.0,NaN,NaN,NaN,NaN,NaN,0,NaN,Neutral,"Cars,Tata",Tamil Nadu
529,19-Sep-2023 02:30PM,NaN,http://twitter.com/indianomicstv/statuses/1704...,NaN,@AshishDarji90 @volklub @TataMotors The most i...,Twitter,@indianomicstv,India,NaN,English,371.0,NaN,NaN,NaN,NaN,NaN,0,1.0,Neutral,"cars,Tata",Maharashtra
1016,14-Sep-2023 08:33AM,NaN,http://twitter.com/iAnurajpK/statuses/17021561...,NaN,RT @dp_karthik: Here is a list of cars that @a...,Twitter,@iAnurajpK,India,NaN,English,6.0,NaN,NaN,NaN,NaN,NaN,0,NaN,Neutral,"cars,Tata",NaN
1520,12-Sep-2023 07:07AM,What makes Karma Cap's Rushabh Sheth bullish o...,https://www.moneycontrol.com/news/business/wha...,Sheth’s big bet on pharma is solid growth in t...,... you watch on TV or online to the real-time...,Moneycontrol.com,Santosh Nair,India,Mumbai,English,20057454.0,3297618.0,16759836.0,11.0,0.0,0.0,0,NaN,Neutral,"cars,Tata",Maharashtra


In [ ]:
df.iloc[0]

Date                                                  28-Sep-2023 02:08PM
Headline                Kia और Hyundai ने वापस बुलाईं अपनी 35 लाख कारे...
URL                     https://hindi.pardaphash.com/kia-and-hyundai-r...
Opening Text            Kia and Hyundai Recalled Cars: वाहन निर्माता क...
Hit Sentence            Kia and Hyundai Recalled Cars: वाहन निर्माता क...
Source                                                         Pardaphash
Influencer                                                            NaN
Country                                                             India
Subregion                                                           Delhi
Language                                                            Hindi
Reach                                                             31307.0
Desktop Reach                                                      1279.0
Mobile Reach                                                      30028.0
Twitter Social Echo                   

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528 entries, 0 to 1527
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  1528 non-null   object 
 1   Headline              710 non-null    object 
 2   URL                   1528 non-null   object 
 3   Opening Text          710 non-null    object 
 4   Hit Sentence          1528 non-null   object 
 5   Source                1528 non-null   object 
 6   Influencer            1065 non-null   object 
 7   Country               1528 non-null   object 
 8   Subregion             321 non-null    object 
 9   Language              1528 non-null   object 
 10  Reach                 1519 non-null   float64
 11  Desktop Reach         710 non-null    float64
 12  Mobile Reach          710 non-null    float64
 13  Twitter Social Echo   710 non-null    float64
 14  Facebook Social Echo  710 non-null    float64
 15  Reddit Social Echo   

In [ ]:
# Checking for Null values
df.isna().sum()

Date                       0
Headline                 818
URL                        0
Opening Text             818
Hit Sentence               0
Source                     0
Influencer               463
Country                    0
Subregion               1207
Language                   0
Reach                      9
Desktop Reach            818
Mobile Reach             818
Twitter Social Echo      818
Facebook Social Echo     818
Reddit Social Echo       818
National Viewership        0
Engagement              1176
Sentiment                  0
Keywords                   0
State                    500
dtype: int64

In [ ]:
# Check for duplicates
df.duplicated().sum()

0

In [ ]:
# categorical columns
cat_cols = df.select_dtypes('object').columns
cat_cols

Index(['Date', 'Headline', 'URL', 'Opening Text', 'Hit Sentence', 'Source',
       'Influencer', 'Country', 'Subregion', 'Language', 'Sentiment',
       'Keywords', 'State'],
      dtype='object')

In [ ]:
# numerical columns
numeric_cols = df.select_dtypes(include=np.number).columns
numeric_cols

Index(['Reach', 'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo',
       'Facebook Social Echo', 'Reddit Social Echo', 'National Viewership',
       'Engagement'],
      dtype='object')

In [ ]:
df['Source'].value_counts().head(50)

Source
Twitter                               818
India Posts English                    89
The Times of India                     32
Cartoq                                 21
Todays Chronic                         14
News Day Express                       13
Team-BHP.com                           13
DNP INDIA                              12
Bulletin Reporter                      12
Hindustan Times                        12
Dainik Jagran                          11
ET Auto.com                            11
Business Standard                      11
GaadiWaadi.com                         10
The Economic Times                      9
News Concerns - Latest News Update      9
Pioneer Newz                            8
Abp Live                                8
The Financial Express                   8
TOI Auto                                8
Nation World News                       8
My Droll                                8
V3Cars                                  8
carandbike                 

In [ ]:
df.isna().sum()

Date                       0
Headline                 818
URL                        0
Opening Text             818
Hit Sentence               0
Source                     0
Influencer               463
Country                    0
Subregion               1207
Language                   0
Reach                      9
Desktop Reach            818
Mobile Reach             818
Twitter Social Echo      818
Facebook Social Echo     818
Reddit Social Echo       818
National Viewership        0
Engagement              1176
Sentiment                  0
Keywords                   0
State                    500
dtype: int64

In [ ]:
df[df['Source']=='Twitter'].isna().sum()

Date                      0
Headline                818
URL                       0
Opening Text            818
Hit Sentence              0
Source                    0
Influencer                0
Country                   0
Subregion               818
Language                  0
Reach                     9
Desktop Reach           818
Mobile Reach            818
Twitter Social Echo     818
Facebook Social Echo    818
Reddit Social Echo      818
National Viewership       0
Engagement              504
Sentiment                 0
Keywords                  0
State                   129
dtype: int64